# Notebook 4: Cifar10 Classification in Pytorch Lightning

In this notebook, we will train an image classifier for the CIFAR-10 dataset, that you already know from exercise 6. Today, however, we will use the PyTorch Lightning framework which makes everything much more convenient!

In case you haven't done yet, you should definitely check out the **PyTorch Lightning Introduction** first!

## (Optional) Mount in Google Colab

In [ ]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_07) is given.
# OPTIONAL: Enable GPU via Runtime --> Change runtime type --> GPU

"""
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/i2dl/exercise_07'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))
"""

### Set up PyTorch environment in colab

For your regular environment this should already have been installed in the previous notebooks.

In [ ]:
# Optional: install correct libraries in google colab
# !python -m pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
# !python -m pip install torchtext==0.12.0 torchaudio==0.11.0
# !python -m pip install tensorboard==2.9.1
# !python -m pip install pytorch-lightning==1.6.0

## Imports

In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
%load_ext autoreload
%autoreload 2

### Get Device
In this exercise, we'll use PyTorch Lightning to build an image classifier for the CIFAR-10 dataset. As you know from exercise 06, processing a large set of images is quite computation extensive. Luckily, with PyTorch we're now able to make use of our GPU to significantly speed things up!

In case you don't have a GPU, you can run this notebook on Google Colab where you can access a GPU for free! 

Of course, you can also run this notebook on your CPU only - though this is definitely not recommended.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Setup TensorBoard
In exercise 07 you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient! Throughout this notebook, feel free to add further logs or visualizations to your TensorBoard!

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs --port 6006

## Define your Network

Do you remember the good old times when we used to implement everything in plain numpy? Luckily, these times are over and we're using PyTorch Lightning which makes everything MUCH easier!

Instead of implementing your own model, solver and dataloader, all you have to do is defining a `LightningModule`.

We've prepared the class `exercise_code/MyPytorchModel` for you, that you'll now finalize to build an image classifier with PyTorch Lightning.

### 0. Dataset & Dataloaders
Check out the function `prepare_data` of the `CIFAR10DataModule` class that loads the dataset, using the class `torchvision.datasets.ImageFolder` (or the previous `MemoryImageFolder` dataset from exercise 3), which is very similar to the class `ImageFolderDataset` that you implemented earlier!

Implement a **transform** to pre-process the raw data (standardize it and convert it to tensors) and assign it to the variable `my_transform`. Note: On the submission server, the normalization as in the notebook 3 on data augmentation will be performed, so please make sure to use the same normalization! For convenience, we added the precomputed normalization values for you. All normalization you are defining here are tailored to your training.

In pytorch-lightning we could also include the dataset and other classes in our model, but a more reasonable way is to define it outside since it usually is used across multiple projects. If you prefer the all-in-one solution, that is great as well, but here we put it separately.

If you want to improve your performance, you can also perform extensive **data augmentation** here!

Also check out the `DataLoader` class that is used to create  `train_dataloader` and `val_dataloader` and that is very similar to your previous implementation of the DataLoader.

### 1. Define your model
Next, let's define your model. Think about a good network architecture. You're completely free here and you can come up with any network you like! (\*)

Have a look at the documentation of `torch.nn` at https://pytorch.org/docs/stable/nn.html to learn how to use use this module to build your network!

Then implement your architecture: initialize it in `__init__()` and assign it to `self.model`. This is particularly easy using `nn.Sequential()` which you only have to pass the list of your layers. 

To make your model customizable and support parameter search, don't use hardcoded hyperparameters - instead, pass them as dictionary `hparams` (here, `n_hidden` is the number of neurons in the hidden layer) when initializing `MyPytorchModel`.

Here's an easy example:

```python
        self.model = nn.Sequential(
            nn.Linear(input_size, self.hparams["n_hidden"]),
            nn.ReLU(),            
            nn.Linear(self.hparams["n_hidden"], num_classes)
        )
```

Have a look at the forward path in `forward(self, x)`, which is so easy, that you don't need to implement it yourself. As PyTorch automatically computes the gradients, that's all we need to do! No need anymore to manually calculate derivatives for the backward paths! :)


____
\* *The size of your final model must be less than 20 MB, which is approximately equivalent to 5 Mio. params. Note that this limit is quite lenient, you will probably need much less parameters!*

*Also, don't use convolutional layers as they've not been covered yet in the lecture and build your network with fully connected layers (```nn.Linear()```)!*

### 2. Training & Validation Step
Have a look at the functions `training_step` and `validation_step` that take a batch as input and calculate the loss. 

### 3. Optimizer
Lastly, implement the function `configure_optimizers()` to define your optimizer. Here, the documentation of `torch.optim`at https://pytorch.org/docs/stable/optim.html might be helpful.

That's it! You've now finalized your `LightningModule` which has (at least) the same functionality as your previous numpy-powered image classifier!

Now let's create an instance of your `MyPytorchModel`.

In [ ]:
from exercise_code.MyPytorchModel import MyPytorchModel, CIFAR10DataModule
# make sure you have downloaded the Cifar10 dataset on root: "../datasets/cifar10", if not, please check exercise 03.
hparams = {}

########################################################################
# TODO: Define your hyper parameters here!                             #
########################################################################

pass

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

# Make sure you downloaded the CIFAR10 dataset already when using this cell
# since we are showcasing the pytorch inhering ImageFolderDataset that
# doesn't automatically download our data. Check exercise 3

# If you want to switch to the memory dataset instead of image folder use
# hparams["loading_method"] = 'Memory'
# The default is hparams["loading_method"] = 'Image'
# You will notice that it takes way longer to initialize a MemoryDataset
# method because we have to load the data points into memory all the time.

# You might get warnings below if you use too few workers. Pytorch uses
# a more sophisticated Dataloader than the one you implemented previously.
# In particular it uses multi processing to have multiple cores work on
# individual data samples. You can enable more than workers (default=2)
# via 
# hparams['num_workers'] = 8

# Set up the data module including your implemented transforms
data_module = CIFAR10DataModule(hparams)
data_module.prepare_data()
# Initialize our model
model = MyPytorchModel(hparams)

Some tests to check whether we'll accept your model:

In [ ]:
from exercise_code.Util import printModelInfo
_ = printModelInfo(model)

## Fit Model with Trainer
Now it's time to train your model. 

Have a look of the documentation of `pl.Trainer` at https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html to find out what arguments you can pass to define your training process. 

Then, start the training with `trainer.fit(model)`.

In [ ]:
import pytorch_lightning as pl
trainer = None

########################################################################
# TODO: Define your trainer!                                           #
########################################################################


pass

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

trainer.fit(model, data_module)

Now that everything is working, feel free to play around with different architectures. As you've seen, it's really easy to define your model or do changes there.

To pass this submission, you'll need **50%** accuracy.

<div class="alert alert-warning">
    <h3>Note: Pytorch vs Pytorch-Lightning</h3>
    <p>At this point you are actually free to submit any pytorch model. A pytorch-lightning model unifies training etc. but it inherently works like a pytorch model when you use `model.forward(blabla)`. For exercise 7 as well as 9-11 you can submit any pytorch model you so desire. In this notebook, we gave you the shell using a pytorch-lightning model since it simplifies everything (i.e., for both you and us on both the code purity as well as the datasets (and, thus, transforms etc.) being part of the model) and you can focus on the notation for now. What you will use later is your own decision and if you prefer to submit pure pytorch code that is cool too! As an exercise for eager people: after submittag and tuning a pytorch-lightning model with the current code, try to submit a pure pytorch version of said code to see what advantages pytorch lightning brings and where you feel you have more control. Good luck!
    </p>
</div>

# Save your model & Report Test Accuracy

When you've done with your **hyperparameter tuning**, have achieved **at least 50% validation accuracy** and are happy with your final model, you can save it here.

Before that, we will check again whether the number of parameters is below 5 Mi and the file size is below 20 MB.

When your final model is saved, we'll lastly report the test accuracy.

In [ ]:
from exercise_code.Util import test_and_save

test_and_save(model, data_module.val_dataloader(), data_module.test_dataloader())

Congrats! You've now finished your first image classifier in PyTorch Lightning! Much easier than in plain numpy, right? Time to get started with some more complex neural networks - see you at the next exercise!

To create a zip file with your submission, run the following cell:

In [ ]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/exercise07')

# Submission Instructions

Congratulations! You've just built your first image classifier with PyTorch Lightning! To complete the exercise, submit your final model to our submission portal - you probably know the procedure by now.

1. Go on [our submission page](https://i2dl.vc.in.tum.de/submission/), register for an account and login. We use your matriculation number and send an email with the login details to the mail account associated. When in doubt, login into tum online and check your mails there. You will get an ID which we need in the next step.
2. Log into [our submission page](https://i2dl.vc.in.tum.de/submission/) with your account details and upload the `zip` file. Once successfully uploaded, you should be able to see the submitted file selectable on the top.
3. Click on this file and run the submission script. You will get an email with your score as well as a message if you have surpassed the threshold.

# Submission Goals

- Goal: Successfully implement a a fully connected NN image classifier for CIFAR-10 with PyTorch Lightning

- Passing Criteria: Similar to the last exercise, there are no unit tests that check specific components of your code. The only thing that's required to pass this optional submission, is your model to reach at least **50% accuracy** on __our__ test dataset. The submission system will show you a number between 0 and 100 which corresponds to your accuracy.

- Submission start: __Dec 08, 2022, 13.00__
- Submission end: __Dec 14, 2022, 15.59__ 
- You can make **$\infty$** submissions until the end of the semester. Remember that this exercise is an __OPTIONAL SUBMISSION__ and will __not__ be counted for the bonus. 

# [Exercise Review](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+7:+PyTorch)

We are always interested in your opinion. Now that you have finished this exercise, we would like you to give us some feedback about the time required to finish the submission and/or work through the notebooks. Please take the short time to fill out our [review form](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+7:+PyTorch) for this exercise so that we can do better next time! :)